In [1]:
from llama_cpp import Llama

llm = Llama.from_pretrained(repo_id="NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF", filename= "*Q4_K_M.gguf", 
                            n_gpu_layers = -1,
                            n_ctx=512,
                            max_tokens=12,
                            n_batch=512,
                            n_threads=6,
                            verbose=True,
                            flash_attn=True)

llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from /home/magsam/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct-gguf/snapshots/c80d904a71b99a3eaeb8d3dbf164166384c09dc3/./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                 

In [39]:
from openai import OpenAI


client = OpenAI(
    base_url="http://localhost:8000/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)

schema = {
    "type": "object",
    "properties": {
        "rating": {
            "type": "number",
            "minimum": 1,
            "maximum": 5,
            "description": "The rating of the joke, from 1 to 5.",
        }
    },
}

joke = "Why did the scarecrow win an award? Because he was outstanding in his field!"

def rate_joke(joke: str):
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": f"You are a joke evaluator that answers in JSON. Here's the json schema you must adhere to:\n<schema>\n{schema}\n</schema>",
        },
        {"role": "user",
            "content": f""" Rate this joke from 1 to 5: \n "{joke}" """},
    ],
    response_format= {"type": "json_object", "schema": schema},
    temperature=0.2,
    top_p=0.9,
    max_tokens=12,
    )

    return completion.choices[0].message.content

rate_joke(joke)

'{"rating": 4}'

In [50]:
import pandas as pd
pretraining_data = pd.read_parquet('../data/processed/pretraining_data.parquet')

# take a subset of 100 jokes to test
pretraining_data = pretraining_data.sample(100)

pretraining_data['rating'] = pretraining_data['cleaned_joke'].apply(rate_joke)

In [51]:
pretraining_data['rating'].value_counts()

rating
{"rating": 3}    53
{"rating": 4}    32
{"rating": 2}    14
{"rating": 5}     1
Name: count, dtype: int64

In [55]:
# Compare the ratings to the original ratings {"rating": 3}  keep only the number
#pretraining_data['rating'] = pretraining_data['rating'].str.extract('(\d+)')
#pretraining_data['rating'] = pretraining_data['rating'].astype(int)

from sklearn.metrics import accuracy_score

accuracy_score(pretraining_data['rating'], pretraining_data['score_class'])


0.23